# Sudoku

El juego del sudoku consiste en completar las celdas vacías con números del 1 al 9, de tal forma que:
* no existan números repetidos en una misma fila,
* no existan números repetidos en una misma columna, y,
* no existan números repetidos en un mismo bloque.

<img src="sudoku_facil.png" width=400>

La tarea de completar un sudoku puede formularse como el problema de encontrar una solución factible para un programa lineal entero. Para ello, empleamos las variables de decisión binarias:

$x_{ij}^r$ : que indican si el número $r \in \{1, \ldots, 9\}$ será colocado en la celda de coordenadas $(i,j)$. Asumiremos que las celdas está numeradas de manera análoga a los elementos de una matriz: la celda superior izquierda tiene coordenadas $(1,1)$, mientras que la celda inferior derecha tiene coordenadas $(9,9,)$.

Notar que, para $i \in \{1, \ldots, 9\}$, la $i$-ésima fila del sudoku contiene las celdas con coordenadas:
$$
F_i = \{(i,j) : 1 \leq j \leq 9 \}.
$$

De manera similar, para $j \in \{1, \ldots, 9\}$, la $j$-ésima columna del sudoku contiene las celdas con coordenadas:
$$
C_j = \{(i,j) : 1 \leq i \leq 9 \}.
$$

Por otra parte, supongamos que los bloques del sudoku están numerados en la siguiente manera:

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 | 5 | 6 |
| 7 | 8 | 9 |

Entonces, para $k \in \{1, \ldots, 9\}$, puede demostrarse que el $k$-ésimo bloque contiene las celdas
$$
B_k = \{(i_0 + i, j_0 + j) : i_0 = \lfloor(k-1)/3\rfloor*3, j_0 = ((k-1)\%3)*3, 1 \leq i,j \leq 3 \}.
$$

Con esta notación, encontrar una asignación válida de números a las casillas del sudoku equivale a encontrar una solución factible para el siguiente sistema de restricciones lineales:

\begin{align*}
& \sum_{r \in R} x_{ij}^r = 1, \quad \forall i \in I, j \in J,\\
& \sum_{(i,j) \in F_i} x_{ij}^r = 1, \quad \forall i \in I, \forall r \in R\\
& \sum_{(i,j) \in C_j} x_{ij}^r = 1, \quad \forall j \in J, \forall r \in R\\
& \sum_{(i,j) \in B_k} x_{ij}^r = 1, \quad \forall k \in K, \forall r \in R\\
&x_{ij}^k \in \{0, 1\}, \quad \forall i \in I, j \in J, r \in R,
\end{align*}
donde  $I = J = K = R = \{1, \ldots, 9\}$ son los conjuntos de índices de filas, columnas y bloques, así como los números posibles a registrar en las celdas.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.


In [ ]:
import gurobipy as gp
from gurobipy import GRB

I = gp.tuplelist(i+1 for i in range(9))
J = gp.tuplelist(j+1 for j in range(9))
K = gp.tuplelist(k+1 for k in range(9))
R = gp.tuplelist(k+1 for k in range(9))

F = {i : gp.tuplelist((i, j) for j in J) for i in I}
C = {j : gp.tuplelist((i, j) for i in I) for j in J}
B = {k : gp.tuplelist((((k-1)//3*3)+i2+1,
                       ((k-1)%3*3) + j2+1) for i2 in range(3) for j2 in range(3)) for k in K}


Como se trata de un problema de factibilidad, no hace falta ingresar una función objetivo en el modelo:

In [ ]:
m = gp.Model('Sudoku')

x = m.addVars(I,J,R, vtype=GRB.BINARY, name='x')

m.addConstrs((x.sum(i,j,'*')==1 for i in I for j in J), "asignacion")
m.addConstrs((gp.quicksum(x[i2,j,r] for (i2,j) in F[i])==1 for i in I for r in R), "filas")
m.addConstrs((gp.quicksum(x[i,j2,r] for (i,j2) in C[j])==1 for j in J for r in R), "columnas")
m.addConstrs((gp.quicksum(x[i,j,r] for (i,j) in B[k])==1 for k in K for r in R), "bloques")


Vamos a fijar ahora las variables a los valores conocidos en el sudoku. Para ello, creamos una lista `fijadas` que contiene tuplas de la forma $(i,j,r)$ para indicar que la celda $(i,j)$ debe contener el valor $r$. Luego usamos esta información para colocar la cota inferior (propiedad `lb`) en 1 para las variables `x[i,j,k]` correspondientes:

In [ ]:
fijadas = [(1,1,9), (1,3,6), (1,5,7), 
           (2,4,2), (2,8,9),
           (3,1,8), (3,2,5), (3,3,1), (3,6,9), (3,7,7),
           (4,1,5), (4,2,6), (4,5,2), (4,7,9), (4,9,3),
           (5,5,1), (5,7,6), (5,8,8),
           (6,3,7), (6,4,6), (6,7,2), (6,9,4),
           (7,2,1), (7,3,9), (7,6,4), (7,8,3), (7,9,8),
           (8,1,7), (8,3,4), (8,4,5), (8,6,8), (8,7,1), (8,8,6),
           (9,2,8), (9,3,5), (9,5,3), (9,7,4), (9,8,7)]

for (i,j,k) in fijadas:
    x[i,j,k].lb=1

Finalmente, llamamos a `optimize()` para encontrar una solución factible al modelo:

In [ ]:
m.optimize()

Usamos el módulo `tabulate` para mostrar esta solución:

In [ ]:
from tabulate import tabulate

sol = [[r for j in J for r in R if x[i,j,r].x>=0.99] for i in I]

for L in sol:
    L.insert(3,'|')
    L.insert(7,'|')
    
sol.insert(3, '---|---|---')
sol.insert(7, '---|---|---')

print(tabulate(sol))
